In [1]:
import pandas as pd
import numpy as np
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
import time

In [ ]:
#load the balanced data
data_balanced = pd.read_csv('UNSW_NB15_balanced_smote.csv')

### CNN Model for Multi-class Classification

In [15]:
# train an lstm model on the data
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.layers import Conv1D
from keras.layers import MaxPooling1D
from keras.layers import Flatten
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn import metrics
X = data_balanced.drop(['attack_cat', 'label'], axis=1)
Y = data_balanced[['attack_cat']]
Y['attack_cat'] = Y['attack_cat'].astype('int')
# Y['label'] = Y['label'].astype('int')
# one hot encode attack_cat
Y = pd.get_dummies(Y,columns=['attack_cat'])

from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.2, random_state = 42)

# convert to numpy array
X_train = np.asarray(X_train).astype(np.float32)
X_test = np.asarray(X_test).astype(np.float32)

# reshape the data for LSTM
X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)
X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)

# convert all to numpy arrays
X_train = np.asarray(X_train).astype(np.float32)
Y_train = np.asarray(Y_train).astype(np.int32)
X_test = np.asarray(X_test).astype(np.float32)
Y_test = np.asarray(Y_test).astype(np.int32)

print(X_train.shape)
# define the model
model = Sequential()
model.add(Conv1D(64, 3,activation="relu",input_shape=(X_train.shape[1], 1)))
model.add(Conv1D(64, 3, activation="relu"))
model.add(MaxPooling1D(pool_size=(2)))
model.add(Conv1D(128, 3, activation="relu"))
model.add(Conv1D(128, 3, activation="relu"))
model.add(MaxPooling1D(pool_size=(2)))
model.add(Flatten())
model.add(Dense(128, activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(6, activation="softmax"))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

mc = ModelCheckpoint('cnn_best_model.h5', monitor='val_accuracy', mode='max', verbose=1, save_best_only=True)

start = time.time()
history = model.fit(X_train, Y_train, epochs=10, batch_size=32, validation_data=(X_test, Y_test), callbacks=[mc])
end = time.time()
print("Time taken to train the model: ", end-start)

C:\Users\Ameya\AppData\Local\Temp\ipykernel_28160\1791411367.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Y['attack_cat'] = Y['attack_cat'].astype('int')


(446400, 196, 1)



Epoch 1/10


13948/13950 [============================>.] - ETA: 0s - loss: 0.5176 - accuracy: 0.8034
Epoch 1: val_accuracy improved from -inf to 0.82214, saving model to cnn_best_model.h5
13950/13950 [==============================] - 190s 13ms/step - loss: 0.5176 - accuracy: 0.8034 - val_loss: 0.4409 - val_accuracy: 0.8221
Epoch 2/10
    6/13950 [..............................] - ETA: 2:47 - loss: 0.4443 - accuracy: 0.8438

C:\Users\Ameya\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


13947/13950 [============================>.] - ETA: 0s - loss: 0.4518 - accuracy: 0.8224
Epoch 2: val_accuracy improved from 0.82214 to 0.82858, saving model to cnn_best_model.h5
13950/13950 [==============================] - 182s 13ms/step - loss: 0.4518 - accuracy: 0.8224 - val_loss: 0.4241 - val_accuracy: 0.8286
Epoch 3/10
13949/13950 [============================>.] - ETA: 0s - loss: 0.4299 - accuracy: 0.8289
Epoch 3: val_accuracy improved from 0.82858 to 0.83886, saving model to cnn_best_model.h5
13950/13950 [==============================] - 204s 15ms/step - loss: 0.4299 - accuracy: 0.8289 - val_loss: 0.4002 - val_accuracy: 0.8389
Epoch 4/10
13949/13950 [============================>.] - ETA: 0s - loss: 0.4152 - accuracy: 0.8347
Epoch 4: val_accuracy improved from 0.83886 to 0.84129, saving model to cnn_best_model.h5
13950/13950 [==============================] - 252s 18ms/step - loss: 0.4152 - accuracy: 0.8347 - val_loss: 0.3897 - val_accuracy: 0.8413
Epoch 5/10
13949/13950 [===

In [16]:
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
from keras.models import load_model
model = load_model('cnn_best_model.h5')
start = time.time()
y_pred = model.predict(X_test)
end = time.time()
print("Time taken to predict: ", end-start)
y_pred = np.argmax(y_pred, axis=1)
Y_test = np.argmax(Y_test, axis=1)

print(classification_report(Y_test, y_pred))
print(accuracy_score(Y_test, y_pred))


3488/3488 [==============================] - 10s 3ms/step
Time taken to predict:  10.98546028137207
              precision    recall  f1-score   support

           0       0.70      0.91      0.79     18639
           1       0.79      0.77      0.78     18534
           2       0.82      0.82      0.82     18672
           3       1.00      0.97      0.99     18497
           4       0.94      0.81      0.87     18721
           5       0.94      0.84      0.89     18537

    accuracy                           0.85    111600
   macro avg       0.87      0.85      0.86    111600
weighted avg       0.87      0.85      0.86    111600

0.8539068100358423


In [17]:
# from confusion matrix calculate accuracy of each class
cm = confusion_matrix(Y_test, y_pred)
cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
print(cm.diagonal())

[0.91045657 0.76977447 0.82208655 0.97432016 0.80887773 0.83853914]
